In [3]:
import os
import xml.etree.ElementTree as ET
from openpyxl import Workbook
from openpyxl.utils import get_column_letter

def obter_dados_nota(arquivo_xml):
    tree = ET.parse(arquivo_xml)
    root = tree.getroot()

    dados_nota = {}
    for elemento in root.iter('{http://www.portalfiscal.inf.br/nfe}vPag'):
        dados_nota['vPag'] = elemento.text.strip() if elemento.text else ''

    for elemento in root.iter('{http://www.portalfiscal.inf.br/nfe}serie'):
        dados_nota['serie'] = elemento.text.strip() if elemento.text else ''

    for elemento in root.iter('{http://www.portalfiscal.inf.br/nfe}nNF'):
        dados_nota['nNF'] = elemento.text.strip() if elemento.text else ''

    for elemento in root.iter('{http://www.portalfiscal.inf.br/nfe}chNFe'):
        dados_nota['chNFe'] = elemento.text.strip() if elemento.text else ''

    xMotivo = root.find('.//{http://www.portalfiscal.inf.br/nfe}xMotivo')
    dados_nota['xMotivo'] = xMotivo.text.strip() if xMotivo is not None else ''

    return dados_nota

# Pasta contendo os arquivos XML
pasta_xml = r"C:\Users\manoe\OneDrive\Área de Trabalho\Users\Partner\Desktop\LA4\xml marco\NFCe"

# Lista os arquivos XML na pasta
arquivos_xml = [arquivo for arquivo in os.listdir(pasta_xml) if arquivo.endswith('.xml')]

# Criação da planilha e adição dos dados
planilha = Workbook()
planilha_ativa = planilha.active

# Percorre os arquivos XML e adiciona os valores na planilha
linha_atual = 2
soma_vPag = 0
for arquivo in arquivos_xml:
    caminho_arquivo_xml = os.path.join(pasta_xml, arquivo)
    dados_nota = obter_dados_nota(caminho_arquivo_xml)
    xMotivo = dados_nota['xMotivo']
    if xMotivo not in ['Evento registrado e vinculado a NF-e']:
        vPag = float(dados_nota.get('vPag', '').replace(',', '.')) if dados_nota.get('vPag') else 0
        soma_vPag += vPag

        tags = list(dados_nota.keys()) + ['Status']
        for coluna, tag in enumerate(tags, start=1):
            coluna_atual = get_column_letter(coluna)
            planilha_ativa.cell(row=1, column=coluna, value=tag)
            planilha_ativa.cell(row=linha_atual, column=coluna, value=dados_nota.get(tag, ''))
        linha_atual += 1

# Adiciona a soma dos valores da coluna A
planilha_ativa.cell(row=linha_atual, column=1, value='Total')
planilha_ativa.cell(row=linha_atual, column=2, value=soma_vPag)

# Salvando a planilha
planilha.save('relatorio_vr.xlsx')
